In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler

In [ ]:
cars = pd.read_csv("./vehicles.csv")
cars.head()

## Cleaning the data
### Removing duplicates and irrelevant columns

In [ ]:
print(f"Length before removing duplicates: {len(cars)}")
clean_cars = cars.drop(["id", "url", "region", "region_url", "VIN", "image_url", "description", "county", "state", "lat", "long", "posting_date"], axis=1)
cars.rename(columns={
    "year": "entry_year",
    "title_status": "vehicle_status",
    "size": "vehicle_size",
    "type": "vehicle_type"
})
clean_cars = clean_cars.drop_duplicates()
print(f"Length after removing duplicates: {len(clean_cars)}")

### Removing all rows that describe the same car but different price

In [ ]:
print(f"Length before removing same cars different price: {len(clean_cars)}")
rows_to_remove = clean_cars[clean_cars.drop("price", axis=1).duplicated(keep=False)].index
clean_cars = clean_cars.drop(rows_to_remove, axis=0)
print(f"Length after removing same cars different price: {len(clean_cars)}")

### Dealing with missing values and outliers

In [ ]:
clean_cars.isnull().any()

In [ ]:
def print_null_values_count_per_column(dataframe):
    end_output = ""
    for column in dataframe.columns:
        end_output += f"{column}: {len(dataframe[dataframe[column].isnull()])},\n"
    end_output = end_output.rstrip(",\n")
    print(end_output)

print_null_values_count_per_column(clean_cars)

In [ ]:
# Car model is essential for predicting price, thus null values are dropped
no_nulls = clean_cars.copy()
no_nulls = no_nulls.dropna(subset="model")

In [ ]:
no_nulls[no_nulls.model.isnull()]

In [ ]:
no_nulls.isnull().any()

In [ ]:
print(len(clean_cars[clean_cars.year.isnull()]))
print(len(clean_cars[clean_cars.manufacturer.isnull()]))
print(len(clean_cars[clean_cars.model.isnull()]))
print(len(clean_cars[clean_cars.condition.isnull()]))
print(len(clean_cars[clean_cars.cylinders.isnull()]))
print(len(clean_cars[clean_cars.fuel.isnull()]))
print(len(clean_cars[clean_cars.odometer.isnull()]))
print(len(clean_cars[clean_cars.title_status.isnull()]))
print(len(clean_cars[clean_cars.transmission.isnull()]))
print(len(clean_cars[clean_cars.drive.isnull()]))
print(len(clean_cars[clean_cars.size.isnull()]))
print(len(clean_cars[clean_cars.type.isnull()]))
print(len(clean_cars[clean_cars.paint_color.isnull()]))

In [ ]:
clean_cars.head()

In [ ]:
no_nulls = clean_cars[~clean_cars.year.isnull()]
no_nulls.year = no_nulls.year.astype(int)
no_nulls.odometer = no_nulls.odometer.fillna(0)
no_nulls.odometer = no_nulls.odometer.astype(int)
no_nulls.head()

In [ ]:
remove_useless = no_nulls.copy()
values_to_replace = remove_useless.model.value_counts()[remove_useless.model.value_counts() < 1000].index
remove_useless.loc[remove_useless.model.isin(values_to_replace), 'model'] = np.nan
remove_useless = remove_useless.rename(columns={'size': 'size1'})
remove_useless.model.value_counts()

In [ ]:
manufacturer_dummies = pd.get_dummies(remove_useless.manufacturer, drop_first=True)
model_dummies = pd.get_dummies(remove_useless.model, drop_first=True)
condition_dummies = pd.get_dummies(remove_useless.condition, drop_first=True)
cylinders_dummies = pd.get_dummies(remove_useless.cylinders, drop_first=True)
fuel_dummies = pd.get_dummies(remove_useless.fuel, drop_first=True)
title_status_dummies = pd.get_dummies(remove_useless.title_status, drop_first=True)
transmission_dummies = pd.get_dummies(remove_useless.transmission, drop_first=True)
drive_dummies = pd.get_dummies(remove_useless.drive, drop_first=True)
size1_dummies = pd.get_dummies(remove_useless.size1, drop_first=True)
type_dummies = pd.get_dummies(remove_useless.type, drop_first=True)
paint_color_dummies = pd.get_dummies(remove_useless.paint_color, drop_first=True)
final_df = pd.concat([remove_useless, manufacturer_dummies, model_dummies, condition_dummies, cylinders_dummies,
                     fuel_dummies, title_status_dummies, transmission_dummies, drive_dummies, size1_dummies,
                     type_dummies, paint_color_dummies], axis="columns")
final_df = final_df.drop(["manufacturer", "model", "condition", "cylinders", "fuel", "title_status", "transmission", 
                         "drive", "size1", "type", "paint_color"], axis=1)
final_df.head()

In [ ]:
final_df = final_df.head(5000)

In [ ]:
final_df.to_csv("please_god.csv")

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn import linear_model
from sklearn.svm import SVR

X = final_df.drop("price", axis=1)
y = final_df.price

models = {
  'linear_regression': {
    'steps': [
        ('scaler', MinMaxScaler()),
        ('regressor', linear_model.LinearRegression())
    ],
    'params': {}
  },
  'suppor_vector_regression': {
    'steps': [
        ('scaler', MinMaxScaler()),
        ('regressor', SVR())
    ],
    'params': {
        'regressor__kernel': ['linear'],
        'regressor__C': [10],
        'regressor__epsilon': [0.1],
        'regressor__gamma': ['auto']
    }
  }
}

scores = []

for model_name, options in models.items():
    print(f"checking model {model_name}")
    pipeline = Pipeline(options["steps"])
    grid_search = GridSearchCV(pipeline, options["params"], cv=5, return_train_score=False, verbose = 4)
    
    grid_search.fit(X, y)
    
    scores.append({
        'model': model_name,
        'best_score': grid_search.best_score_,
        'best_params': grid_search.best_params_
    })

df = pd.DataFrame(scores,columns=['model','best_score','best_params'])
df

In [ ]:
model = SVR(kernel="linear", C=10, epsilon=0.1, gamma="auto")
grid_search = GridSearchCV(model, [], cv=5, return_train_score=False, verbose = 4)
grid_search.fit(X, y)
grid_search.best_score_

In [ ]:
df = pd.read_csv("./Airbnb-US-2023.csv")
# Give some columns clearer/shorter names
df = df.rename(columns={
    "price": "price_per_night",
    "number_of_reviews": "nr_of_reviews",
    "last_review": "last_review_date",
    "calculated_host_listings_count": "nr_of_listings_by_host",
    "availability_365": "days_per_year_available",
    "number_of_reviews_ltm": "nr_of_reviews_last_twelve_months"
})
df.head()

## Visualizing the locations
### Use the output of the function below with google my maps to plot a number of locations unto a map

In [ ]:
def export_locations_to_google_my_maps(dataframe, number_of_locations = 10000):
    """Extracts latitude, longitude, name, and room_type from the dataframe provided.
    The locations are reduced until their number is close to number_of_locations.
    The result is grouped based on room_type, each group is exported to multiple csv files of max length = 2000.
    These are to be imported on Google My Maps for plotting."""
    
    dataframe = dataframe[["latitude", "longitude", "name", "room_type"]]
    
    entire_apartment_df = dataframe[dataframe["room_type"] == "Entire home/apt"].drop("room_type", axis=1)
    private_room_df = dataframe[dataframe["room_type"] == "Private room"].drop("room_type", axis=1)
    shared_room_df = dataframe[dataframe["room_type"] == "Shared room"].drop("room_type", axis=1)
    hotel_room_df = dataframe[dataframe["room_type"] == "Hotel room"].drop("room_type", axis=1)
    
    locations_to_eliminate = len(dataframe) - number_of_locations
    entire_apartment_ratio = len(entire_apartment_df) / len(dataframe)
    private_room_ratio = len(private_room_df) / len(dataframe)
    shared_room_ratio = len(shared_room_df) / len(dataframe)
    hotel_room_ratio = len(hotel_room_df) / len(dataframe)
    
    entire_apartment_locations_to_delete = int(entire_apartment_ratio * locations_to_eliminate)
    private_room_locations_to_delete = int(private_room_ratio * locations_to_eliminate)
    shared_room_locations_to_delete = int(shared_room_ratio * locations_to_eliminate)
    hotel_room_locations_to_delete = int(hotel_room_ratio * locations_to_eliminate)
    
    entire_apartment_locations_to_keep = len(entire_apartment_df) - entire_apartment_locations_to_delete
    private_room_locations_to_keep = len(private_room_df) - private_room_locations_to_delete
    shared_room_locations_to_keep = len(shared_room_df) - shared_room_locations_to_delete
    hotel_room_locations_to_keep = len(hotel_room_df) - hotel_room_locations_to_delete
    
    entire_apartment_df_shortened = entire_apartment_df.sample(entire_apartment_locations_to_keep)
    private_room_df_shortened = private_room_df.sample(private_room_locations_to_keep)
    shared_room_df_shortened = shared_room_df.sample(shared_room_locations_to_keep)
    hotel_room_df_shortened = hotel_room_df.sample(hotel_room_locations_to_keep)
    
    entire_apartment_dfs = np.array_split(entire_apartment_df_shortened, len(entire_apartment_df_shortened) // 2000 + 1)
    for i, df_chunk in enumerate(entire_apartment_dfs):
        df_chunk.to_csv(f"google-my-maps-csvs/apartment{i}.csv")
    
    private_room_dfs = np.array_split(private_room_df_shortened, len(private_room_df_shortened) // 2000 + 1)
    for i, df_chunk in enumerate(private_room_dfs):
        df_chunk.to_csv(f"google-my-maps-csvs/private{i}.csv")
    
    shared_room_dfs = np.array_split(shared_room_df_shortened, len(shared_room_df_shortened) // 2000 + 1)
    for i, df_chunk in enumerate(shared_room_dfs):
        df_chunk.to_csv(f"google-my-maps-csvs/shared{i}.csv")
    
    hotel_room_dfs = np.array_split(hotel_room_df_shortened, len(hotel_room_df_shortened) // 2000 + 1)
    for i, df_chunk in enumerate(hotel_room_dfs):
        df_chunk.to_csv(f"google-my-maps-csvs/hotel{i}.csv")

## Cleaning the data
### Removing duplicates and irrelevant columns

In [ ]:
print(f"Length before removing duplicates: {len(df)}")
relevant_df = df.drop(["id"], axis=1)
relevant_df = relevant_df.drop_duplicates()
# We drop neighbourhood_group because we already have the neighbourhood column which provides more information
relevant_df = relevant_df.drop(["host_id", "host_name", "latitude", "longitude", "neighbourhood_group"], axis=1)
print(f"Length after removing duplicates: {len(relevant_df)}")

### Dealing with missing values and outliers

In [ ]:
relevant_df.isnull().any()

In [ ]:
print(f"Length before removing irrelevant columns: {len(relevant_df)}")
non_null_df = relevant_df.copy()
non_null_df = non_null_df.dropna(subset=["name"])
non_null_df.last_review_date = non_null_df.last_review_date.fillna(0)
non_null_df.reviews_per_month = non_null_df.reviews_per_month.fillna(0)
non_null_df.loc[non_null_df.last_review_date != 0, "last_review_date"] = pd.to_datetime(non_null_df[non_null_df.last_review_date != 0].last_review_date).apply(lambda x: int(x.timestamp()))
print(f"Length after removing irrelevant columns: {len(non_null_df)}")

In [ ]:
# Create a histogram of every column that could have outliers to see which ones have outliers
# Alongside there will be plotted 2 vertical lines representing the bounds for eliminating outliers
columns_used_for_checking_outliers = non_null_df.drop(["name", "neighbourhood", "room_type", "city"], axis=1).columns

rows_of_subplots = int(len(columns_used_for_checking_outliers) / 2)
columns_of_subplots = 2

fig, axes = plt.subplots(rows_of_subplots, columns_of_subplots, figsize=(14, 10))
fig.subplots_adjust(hspace=0.9, wspace=0.2)
axes = axes.flatten()

for subplot_index, column_name in enumerate(columns_used_for_checking_outliers):
    ax = axes[subplot_index]
    ax.hist(non_null_df[column_name], bins=20, rwidth=0.8)
    
    mean = non_null_df[column_name].mean()
    standard_deviation = non_null_df[column_name].std()
    
    lower_bound = mean - (2 * standard_deviation)
    upper_bound = mean + (2 * standard_deviation)

    ax.axvline(x=lower_bound, color='b')
    ax.axvline(x=upper_bound, color='b')
    
    # most subplots need to be scaled logarithmically because there is one range of values that is much more frequent
    # than any other, thus the subplot won't give too much information unless it is scaled
    if column_name != "days_per_year_available":
        ax.set_yscale("log")
    
    ax.set_xlabel(column_name)
    ax.set_ylabel("frequency")
    ax.set_title(f"Distribution of {column_name}")
    
plt.show()

In [ ]:
# last_review_date and days_per_year_available will be excluded from outlier removal because they don't have any outliers

columns_used_for_removing_outliers = non_null_df.drop(["name", "neighbourhood", "room_type", "city", "last_review_date", "days_per_year_available"], axis=1).columns

print(f"Length before removing outliers: {len(non_null_df)}\n")

no_outliers_df = non_null_df.copy()

for column_name in columns_used_for_removing_outliers:
    mean = no_outliers_df[column_name].mean()
    standard_deviation = no_outliers_df[column_name].std()
    
    lower_bound = mean - (2 * standard_deviation)
    upper_bound = mean + (2 * standard_deviation)
    
    percentage_removed = round((((no_outliers_df[column_name] < lower_bound) | (no_outliers_df[column_name] > upper_bound)).sum() / len(no_outliers_df)) * 100, 2)

    print(f"For column {column_name}, removing a percentage of {percentage_removed}% values.")
    no_outliers_df = no_outliers_df[(lower_bound <= no_outliers_df[column_name]) & (no_outliers_df[column_name] <= upper_bound)]

print(f"\nLength after removing outliers: {len(no_outliers_df)}")

## Feature engineering

### Extracting meaningful words from name column

In [ ]:
name_meaningful_words_df = no_outliers_df.copy()
name_meaningful_words_df.name = name_meaningful_words_df.name.str.lower()
# Extract number of bedrooms in a separate column
name_meaningful_words_df["nr_of_bedrooms"] = name_meaningful_words_df.name.str.extract('(\d+)\s*-?\+?(?:br|bd|bed)')
name_meaningful_words_df["has_nr_of_bedrooms"] = ~name_meaningful_words_df["nr_of_bedrooms"].isnull()
name_meaningful_words_df["nr_of_bedrooms"] = name_meaningful_words_df["nr_of_bedrooms"].fillna(0)

def most_frequent_words_in_name_column(dataframe):
    # filter anything that is not helpful, or any words that are related to location
    filtered_strings = {"in", "private", "to", "bedroom", "the", "room", "home", "with", "apartment", "near", "and", "house", "of", 
    "a", "from", "bed", "apt", "bath", "close", "view", "suite", "on", "walk", "location", "for", "east", "by", "at", "hollywood", 
    "city", "one", "austin", "unit", "br", "nr", "west", "s", "d", "dt", "no", "bd", "b", "la", "full", "brooklyn", "san", "brand", 
    "hill", "steps", "minutes", "bathroom", "strip", "vegas", "manhattan", "prime", "las", "hills", "south", "mins", "hot", "guest", 
    "two", "free", "min", "hotel", "nyc", "place", "w", "floor", "stay", "nashville", "space", "shared", "away", "north", "entire", 
    "beds", "sleeps", "bay", "williamsburg", "side", "living", "kitchen", "rental", "square", "district", "located", "street", "area", "your", 
    "upper", "neighborhood", "pet", "st", "bdrm", "venice", "los", "style", "santa", "bedrooms", "blocks", "heights", "diego", 
    "block", "next", "miles", "capitol", "long", "centre", "dtla", "top", "broadway", "seattle", "mission", "all"}

    word_frequencies = dataframe.name.str.split('\\W+', expand=True).stack().value_counts()
    word_frequencies = word_frequencies[(word_frequencies >= 1000) & word_frequencies.index.str.isalpha()]
    word_frequencies = word_frequencies[~word_frequencies.index.isin(filtered_strings)]
    
    return word_frequencies

most_frequent_words_in_name_column(name_meaningful_words_df)

In [ ]:
# Frequent words have to be manually extracted because similar words were grouped together to reduce number of columns
# And some were removed due to not being useful
meaningful_words_with_column_name = [
    ["peaceful", ["quiet", "oasis", "retreat", "getaway", "peaceful"]],
    ["luxurios", ["luxury", "luxurious", "resort", "deluxe"]],
    ["lovely", ["lovely", "charming", "cute", "nice"]],
    ["spacious", ["spacious", "large", "huge", "big"]],
    ["comfortable", ["cozy", "comfy", "comfortable"]],
    ["central", ["downton", "heart", "central", "midtown"]],
    ["amazing", ["great", "amazing", "best", "perfect", "paradise"]],
    ["beautiful", ["beautiful", "gorgeous", "stunning"]],
    ["bright", ["sunny", "bright", "cheerful"]],
    ["renovated", ["renovated", "remodeled", "newly"]],
    ["backyard", ["garden", "backyard", "yard"]],
    ["beach", ["beach", "ocean"]],
    ["pool", ["pool", "tub"]],
    ["patio", ["deck", "patio"]],
    ["stylish", ["stylish", "chic"]],
    ["convenient", ["convenient", "everything"]],
    ["townhouse", ["townhouse", "townhome"]],
    ["urban", ["urban", "town",]],
    ["modern", ["modern", "new",]],
    ["village", ["village"]],
    ["cottage", ["cottage"]],
    ["studio", ["studio"]],
    ["condo", ["condo"]],
    ["bungalow", ["bungalow"]],
    ["villa", ["villa"]],
    ["penthouse", ["penthouse"]],
    ["duplex", ["duplex"]],
    ["loft", ["loft"]],
    ["king", ["king"]],
    ["queen", ["queen"]],
    ["master", ["master"]],
    ["gym", ["gym"]],
    ["entrance", ["entrance"]],
    ["balcony", ["balcony"]],
    ["rooftop", ["rooftop"]],
    ["mid", ["mid"]],
    ["victorian", ["victorian"]],
    ["historic", ["historic"]],
    ["gem", ["gem"]],
    ["clean", ["clean"]],
    ["furnished", ["furnished"]],
    ["family", ["family"]],
    ["friendly", ["friendly"]],
    ["views", ["views"]],
    ["valley", ["valley"]],
    ["park", ["park"]],
    ["lake", ["lake"]],
    ["waterfront", ["waterfront"]],
    ["vacation", ["vacation"]],
    ["spa", ["spa"]],
    ["heated", ["heated"]],
    ["airport", ["airport"]],
    ["parking", ["parking"]],
    ["wifi", ["wifi"]]
]

# for each meaningful word column (column_name) create a new column in the dataframe with its name (e.g. luxurious)
# split each row's name with delimiters being whitespace or special characters
# if any word in the split name is inside a list of words (values_to_search)
# set the value for the current column to True, otherwise set to False
for column_name, values_to_search in meaningful_words_with_column_name:
    name_meaningful_words_df[column_name] = name_meaningful_words_df.name.str.split('\\W+').apply(lambda cur_word: any(value_to_search in cur_word for value_to_search in values_to_search))

name_meaningful_words_df = name_meaningful_words_df.drop("name", axis=1)
name_meaningful_words_df.head()

### Checking if city column needs any modifications

In [ ]:
# There aren't too many cities, so the values will remain unmodified until they are one-hot encoded
name_meaningful_words_df.city.value_counts()

In [ ]:
reduced_frequency_values_removed_df = name_meaningful_words_df.copy()
values_to_replace = reduced_frequency_values_removed_df.city.value_counts()[reduced_frequency_values_removed_df.city.value_counts() < 5000].index
reduced_frequency_values_removed_df.loc[reduced_frequency_values_removed_df.city.isin(values_to_replace), 'city'] = 'Other'
reduced_frequency_values_removed_df.city.value_counts()

In [ ]:
# There are many values with a frequency of one, everything with a frequency < 1000 will be converted to 'Other'
name_meaningful_words_df.neighbourhood.value_counts()

In [ ]:
values_to_replace = reduced_frequency_values_removed_df.neighbourhood.value_counts()[reduced_frequency_values_removed_df.neighbourhood.value_counts() < 1000].index
reduced_frequency_values_removed_df.loc[reduced_frequency_values_removed_df.neighbourhood.isin(values_to_replace), 'neighbourhood'] = 'Other'
reduced_frequency_values_removed_df.neighbourhood.value_counts()

### One-hot encoding

In [ ]:
# One-hot encoding the room_type, city and neighbourhood columns
room_type_dummies = pd.get_dummies(reduced_frequency_values_removed_df.room_type, drop_first=True)
city_dummies = pd.get_dummies(reduced_frequency_values_removed_df.city, drop_first=True)
neighbourhood_dummies = pd.get_dummies(reduced_frequency_values_removed_df.neighbourhood, drop_first=True)
final_df = pd.concat([reduced_frequency_values_removed_df, room_type_dummies, neighbourhood_dummies, city_dummies], axis="columns")
final_df = final_df.drop(["neighbourhood", "room_type", "city"], axis=1)
final_df.head()

In [ ]:
final_df = final_df.head(10000)

In [ ]:
len(final_df)

In [ ]:
final_df['price_per_night'] = pd.cut(final_df['price_per_night'], bins=[-1, 90, 150, 257, float('inf')],
                           labels=[0, 1, 2, 3])
final_df.head()

In [ ]:
final_df.to_csv("df_full_size.csv")

In [ ]:
len(final_df[final_df.has_nr_of_bedrooms])

In [ ]:
final_df.to_csv("loosing_my_neurons.csv")

In [ ]:
final_df[final_df.price_per_night1.isnull()]

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn import linear_model
from sklearn.svm import SVC

X = final_df.drop("price_per_night", axis=1)
y = final_df.price_per_night

models = {
  'linear_regression': {
    'steps': [
        ('scaler', MinMaxScaler()),
        ('regressor', linear_model.LogisticRegression(max_iter=10000))
    ],
    'params': {}
  }
}

scores = []

for model_name, options in models.items():
    print(f"checking model {model_name}")
    pipeline = Pipeline(options["steps"])
    grid_search = GridSearchCV(pipeline, options["params"], cv=5, return_train_score=False, verbose = 4)
    
    grid_search.fit(X, y)
    
    scores.append({
        'model': model_name,
        'best_score': grid_search.best_score_,
        'best_params': grid_search.best_params_
    })

df = pd.DataFrame(scores,columns=['model','best_score','best_params'])
df

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn import linear_model
from sklearn.svm import SVR

X = final_df.drop("price_per_night", axis=1)
y = final_df.price_per_night

models = {
  'linear_regression': {
    'steps': [
        ('scaler', MinMaxScaler()),
        ('regressor', linear_model.LinearRegression())
    ],
    'params': {}
  }
}

scores = []

for model_name, options in models.items():
    print(f"checking model {model_name}")
    pipeline = Pipeline(options["steps"])
    grid_search = GridSearchCV(pipeline, options["params"], cv=5, return_train_score=False, verbose = 4)
    
    grid_search.fit(X, y)
    
    scores.append({
        'model': model_name,
        'best_score': grid_search.best_score_,
        'best_params': grid_search.best_params_
    })

df = pd.DataFrame(scores,columns=['model','best_score','best_params'])
df

## Testing ml models

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn import linear_model
from sklearn.svm import SVR

X = final_df.drop("price_per_night", axis=1)
y = final_df.price_per_night

models = {
  'linear_regression': {
    'steps': [
        ('scaler', MinMaxScaler()),
        ('regressor', linear_model.LinearRegression())
    ],
    'params': {}
  },
  'suppor_vector_regression': {
    'steps': [
        ('scaler', MinMaxScaler()),
        ('regressor', SVR())
    ],
    'params': {
        'regressor__kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
        'regressor__C': [0.1, 10],
        'regressor__epsilon': [0.1, 0.001],
        'regressor__gamma': ['scale', 'auto']
    }
  }
}

scores = []

for model_name, options in models.items():
    print(f"checking model {model_name}")
    pipeline = Pipeline(options["steps"])
    grid_search = GridSearchCV(pipeline, options["params"], cv=5, return_train_score=False, verbose = 4)
    
    grid_search.fit(X, y)
    
    scores.append({
        'model': model_name,
        'best_score': grid_search.best_score_,
        'best_params': grid_search.best_params_
    })

df = pd.DataFrame(scores,columns=['model','best_score','best_params'])
df